# Pipelines subpackage

The `opr.pipelines` subpackage contains ready-to-use pipelines for model inference.

## Imports & functions

In [1]:
%load_ext autoreload
%autoreload 2

In [38]:
from time import time

from hydra.utils import instantiate
import numpy as np
from omegaconf import OmegaConf
from scipy.spatial.transform import Rotation

try:
    from geotransformer.utils.registration import compute_registration_error
    from geotransformer.utils.pointcloud import get_transform_from_rotation_translation
except ImportError:
    print("WARNIGN: geotransformer not installed, registration error will not be computed")

from opr.datasets.itlp import ITLPCampus
from opr.pipelines.place_recognition import PlaceRecognitionPipeline
from opr.pipelines.registration import PointcloudRegistrationPipeline
from opr.pipelines.localization import LocalizationPipeline


def pose_to_matrix(pose):
    """From the 6D poses in the [tx ty tz qx qy qz qw] format to 4x4 pose matrices."""
    position = pose[:3]
    orientation_quat = pose[3:]
    rotation = Rotation.from_quat(orientation_quat)
    pose_matrix = np.eye(4)
    pose_matrix[:3,:3] = rotation.as_matrix()
    pose_matrix[:3,3] = position
    return pose_matrix


def compute_error(estimated_pose, gt_pose):
    """For the 6D poses in the [tx ty tz qx qy qz qw] format."""
    estimated_pose = pose_to_matrix(estimated_pose)
    gt_pose = pose_to_matrix(gt_pose)
    error_pose = np.linalg.inv(estimated_pose) @ gt_pose
    dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
    r = Rotation.from_matrix(error_pose[:3, :3])
    rotvec = r.as_rotvec()
    angle_error = (np.sum(rotvec**2)**0.5) * 180 / np.pi
    angle_error = abs(90 - abs(angle_error-90))
    return dist_error, angle_error

def compute_translation_error(gt_pose, pred_pose):
    """For the 4x4 pose matrices."""
    gt_trans = gt_pose[:3, 3]
    pred_trans = pred_pose[:3, 3]
    error = np.linalg.norm(gt_trans - pred_trans)
    return error

def compute_rotation_error(gt_pose, pred_pose):
    """For the 4x4 pose matrices."""
    gt_rot = Rotation.from_matrix(gt_pose[:3, :3])
    pred_rot = Rotation.from_matrix(pred_pose[:3, :3])
    error = Rotation.inv(gt_rot) * pred_rot
    error = error.as_euler('xyz', degrees=True)
    error = np.linalg.norm(error)
    return error

def compute_absolute_pose_error(gt_pose, pred_pose):
    """For the 4x4 pose matrices."""
    rotation_error = compute_rotation_error(gt_pose, pred_pose)
    translation_error = compute_translation_error(gt_pose, pred_pose)
    return rotation_error, translation_error


## Usage example - Place Recognition Pipeline

### Config

In [6]:
DATABASE_TRACK_DIR = "/home/docker_opr/Datasets/ITLP-Campus-old-data/00_2023-02-10"
QUERY_TRACK_DIR = "/home/docker_opr/Datasets/ITLP-Campus-old-data/01_2023-02-21"

DEVICE = "cuda"

MODEL_CONFIG_PATH = "../configs/model/place_recognition/minkloc3d.yaml"
WEIGHTS_PATH = "../weights/place_recognition/minkloc3d_nclt.pth"

#### Init query dataset

The pipeline infer method accepts an input in the format of dictionary with keys in the following format:
- `"image_{camera_name}"` for images from cameras,
- `"mask_{camera_name}"` for semantic segmentation masks,
- `"pointcloud_lidar_coords"` for pointcloud coordinates from lidar,
- `"pointcloud_lidar_feats"` for pointcloud features from lidar.

The data type of all values are `torch.Tensor`.

You can load and preprocess the data manually, but in this example we will use the `opr.datasets.itlp.ITLPCampus` ` dataset class. 

In [7]:
query_dataset = ITLPCampus(
    dataset_root=QUERY_TRACK_DIR,
    sensors=["lidar"],
    mink_quantization_size=0.5,
    load_semantics=False,
    load_text_descriptions=False,
    load_text_labels=False,
    load_aruco_labels=False,
    indoor=False,
)


#### Initialize model

We will use hydra's `instantiate` function to initialize the model. The model is a `MinkLoc3D` - a simple LiDAR-only architecture.

In [8]:
model_config = OmegaConf.load(MODEL_CONFIG_PATH)
model = instantiate(model_config)

#### Initialize pipeline

The minimum requirement to initialize the `PlaceRecognitionPipeline` is that the database directory should contain the `index.faiss` file and the `track.csv` file.

The `index.faiss` file is a Faiss index, which contains the descriptors of the database. The `track.csv` file contains the metadata of the database, including the id and the pose of the descriptors.

The details on how to create the database are described in the [build_database.ipynb](./build_database.ipynb) notebook.

Note that the actual data are not required, as the pipeline will only load the `index.faiss` and the `track.csv` file. This can be useful in the real-world scenario, where the database size is too large to be stored on the local machine.

In [9]:
pipe = PlaceRecognitionPipeline(
    database_dir=DATABASE_TRACK_DIR,
    model=model,
    model_weights_path=WEIGHTS_PATH,
    device=DEVICE,
)


#### Run inference

In [10]:
sample_data = query_dataset[5]
sample_pose_gt = sample_data.pop("pose")  # removing those keys are not necessary, we just
sample_data.pop("idx")                    # want to simulate that we pass the data without GT information :)
print(f"sample_data.keys() = {sample_data.keys()}")
sample_output = pipe.infer(sample_data)
print(f"sample_output.keys() = {sample_output.keys()}")
print(f"sample_output['idx'] = {sample_output['idx']}")
print(f"pose = {sample_output['pose']}")
print(f"pose_gt = {sample_pose_gt.numpy()}")
dist_error, angle_error = compute_error(sample_output["pose"], sample_pose_gt.numpy())
print(f"dist_error = {dist_error}, angle_error = {angle_error}")


sample_data.keys() = dict_keys(['pointcloud_lidar_coords', 'pointcloud_lidar_feats'])
sample_output.keys() = dict_keys(['idx', 'pose', 'descriptor'])
sample_output['idx'] = 6
pose = [ 6.40806188e-01 -6.96985360e+00 -2.62666965e+00  2.23428939e-03
  1.21724469e-02  3.57296189e-01  9.33909135e-01]
pose_gt = [-0.3476145  -7.5303183  -1.8204867  -0.01149435  0.01080806  0.3418056
  0.9396382 ]
dist_error = 1.3932074823071565, angle_error = 2.466076215796548


## Usage example - Pointcloud Registration Pipeline

### Config

In [11]:
TRACK_DIR = QUERY_TRACK_DIR

REGISTRATION_MODEL_CONFIG_PATH = "../configs/model/registration/geotransformer_kitti.yaml"
REGISTRATION_WEIGHTS_PATH = "../weights/registration/geotransformer_kitti.pth"

### Init dataset

For the demonstration purpose, we will use the `opr.datasets.itlp.ITLPCampus` dataset class to load the data.

We will instantiate only one track and test the registration performance by evaluating the transformation between the two consecutive frames.

In [12]:
dataset = ITLPCampus(
    dataset_root=TRACK_DIR,
    sensors=["lidar"],
    mink_quantization_size=0.5,
    load_semantics=False,
    load_text_descriptions=False,
    load_text_labels=False,
    load_aruco_labels=False,
    indoor=False,
)

### Init model

In [13]:
geotransformer = instantiate(OmegaConf.load(REGISTRATION_MODEL_CONFIG_PATH))


### Init Pipeline

In [14]:
registration_pipe = PointcloudRegistrationPipeline(
    model=geotransformer,
    model_weights_path=REGISTRATION_WEIGHTS_PATH,
    device="cuda",  # the GeoTransformer currently only supports CUDA
    voxel_downsample_size=0.3,  # recommended for geotransformer_kitti configuration
)

### Run inference

In [15]:
i = 3
db_pc = dataset[i-1]["pointcloud_lidar_coords"]
query_pc = dataset[i]["pointcloud_lidar_coords"]
db_pose = pose_to_matrix(dataset[i-1]["pose"])
query_pose = pose_to_matrix(dataset[i]["pose"])
# we want to find the transformation from the "database" pose to the "query" pose
gt_transformation = np.linalg.inv(db_pose) @ query_pose

estimated_transformation = registration_pipe.infer(query_pc, db_pc)

print(f"gt_transformation = \n{gt_transformation}\n")
print(f"estimated_transformation = \n{estimated_transformation}\n")

rre, rte = compute_registration_error(gt_transformation, estimated_transformation)
print(f"Relative Rotation Error (RRE) = {rre:0.3f}\nRelative Translation Error (RTE) = {rte:0.3f}\n")

gt_transformation = 
[[ 8.91628375e-01 -4.52424470e-01 -1.76334775e-02  5.29986862e+00]
 [ 4.52764020e-01  8.91106946e-01  3.05475466e-02  1.28405765e+00]
 [ 1.89285673e-03 -3.52208635e-02  9.99377760e-01  1.87355578e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

estimated_transformation = 
[[ 0.89398414 -0.44688743 -0.03291935  5.3991265 ]
 [ 0.44747856  0.89419794  0.01314684  1.2985258 ]
 [ 0.02356126 -0.02648378  0.9993715   0.20784616]
 [ 0.          0.          0.          1.        ]]

Relative Rotation Error (RRE) = 1.378
Relative Translation Error (RTE) = 0.102



/home/docker_opr/OpenPlaceRecognition/third_party/GeoTransformer/geotransformer/modules/geotransformer/superpoint_matching.py:44: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ref_sel_indices = corr_indices // matching_scores.shape[1]


In [16]:
print(f"gt_pose = \n{query_pose}")
print(f"optimized_pose = \n{db_pose @ estimated_transformation}")

gt_pose = 
[[  0.81298777  -0.58208961   0.01491904  -8.01054096]
 [  0.58153938   0.81297861   0.02962627 -14.29785252]
 [ -0.02937401  -0.01540979   0.9994497   -1.86421597]
 [  0.           0.           0.           1.        ]]
optimized_pose = 
[[ 8.16909498e-01 -5.76760468e-01  2.42963783e-03 -7.91389234e+00]
 [ 5.76712639e-01  8.16884243e-01  1.01272679e-02 -1.42685991e+01]
 [-7.82573585e-03 -6.87186963e-03  9.99945754e-01 -1.84734572e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Usage example - Localization Pipeline

Localization pipeline combines the place recognition and the pointcloud registration pipeline.

### Init database dataset

In [35]:
db_dataset = ITLPCampus(
    dataset_root=DATABASE_TRACK_DIR,
    sensors=["lidar"],
)

### Init Localization Pipeline

Here we will use the PlaceRecognition and Registration pipelines that we have initialized in the previous examples.

In [36]:
localization_pipe = LocalizationPipeline(
    place_recognition_pipeline=pipe,
    registration_pipeline=registration_pipe,
    db_dataset=db_dataset,
)

### Test the pipeline

In [39]:
PR_MATCH_THRESHOLD = 25.0
pr_matches = []
rre_list = []
rte_list = []
times = []

for sample in query_dataset:
    gt_pose = sample.pop("pose")
    gt_pose = get_transform_from_rotation_translation(Rotation.from_quat(gt_pose[3:]).as_matrix(), gt_pose[:3])

    start_time = time()
    pipe_out = localization_pipe.infer(sample)
    times.append(time() - start_time)

    db_match_pose = pipe_out["db_match_pose"]
    db_match_pose = get_transform_from_rotation_translation(Rotation.from_quat(db_match_pose[3:]).as_matrix(), db_match_pose[:3])
    estimated_pose = pipe_out["estimated_pose"]
    estimated_pose = get_transform_from_rotation_translation(Rotation.from_quat(estimated_pose[3:]).as_matrix(), estimated_pose[:3])

    _, db_match_distance = compute_registration_error(gt_pose, db_match_pose)
    pr_matched = db_match_distance <= PR_MATCH_THRESHOLD
    pr_matches.append(pr_matched)

    rre, rte = compute_registration_error(gt_pose, estimated_pose)
    rre_list.append(rre)
    rte_list.append(rte)

/home/docker_opr/OpenPlaceRecognition/third_party/GeoTransformer/geotransformer/modules/geotransformer/superpoint_matching.py:44: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ref_sel_indices = corr_indices // matching_scores.shape[1]
/home/docker_opr/OpenPlaceRecognition/third_party/GeoTransformer/geotransformer/modules/geotransformer/superpoint_matching.py:44: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), o

In [40]:
print(f"Place Recognition R@1 = {np.mean(pr_matches):0.3f}")
print(f"Localization Mean RRE = {np.mean(rre_list):0.3f}")
print(f"Localization Mean RTE = {np.mean(rte_list):0.3f}")

print(f"Localization Median RRE = {np.median(rre_list):0.3f}")
print(f"Localization Median RTE = {np.median(rte_list):0.3f}")

print(f"Mean Time = {(np.mean(times) * 1000):0.2f} ms")

Place Recognition R@1 = 0.915
Localization Mean RRE = 13.566
Localization Mean RTE = 9.688
Localization Median RRE = 2.017
Localization Median RTE = 0.927
Mean Time = 509.46 ms
